In [85]:
%load_ext autoreload
%autoreload 2
# Enable imports form top-level of project (edit top_level_path accordingly)
import os
import sys
top_level_path = os.path.abspath(os.path.join('..'))
if top_level_path not in sys.path:
	sys.path.append(top_level_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [86]:
from pyqubo import Binary, Array, Num, LogEncInteger
import numpy as np
from dimod import ExactSolver
import neal
from longestpath import gen_average_degree_directed, gen_planted_path, StandardGraph
from typing import List, Tuple, Callable
from math import log2, floor, ceil

In [87]:
def make_tail_graph(graph : StandardGraph, tail_length : int) -> StandardGraph:

	if tail_length == 0:
		tail_graph = StandardGraph(0, [])
		tail_graph.vertices += graph.vertices
		tail_graph.edges = graph.edges.copy()
		return tail_graph

	tail_graph = StandardGraph(0, [])
	tail_graph.vertices += graph.vertices + tail_length
	tail_graph.edges = graph.edges.copy()
	
	for i in range(0,graph.vertices):
		tail_graph.edges.append((i,graph.vertices))
	
	for i in range(graph.vertices , graph.vertices + tail_length - 1):
		tail_graph.edges.append((i, i+1))

	return tail_graph


def make_vertex_edges(graph : StandardGraph) -> List[List[int]] :
	out = [[] for i in range(graph.vertices)]

	for i in range(len(graph.edges)):
		(v_1, v_2) = graph.edges[i]
		
		#out[v_1].append((i,v_1,v_2))
		#out[v_2].append((i,v_1,v_2))
		out[v_1].append(i)
		out[v_2].append(i)
	
	return out

def make_vertex_edges_ingoing(graph : StandardGraph) -> List[List[int]] :
	out = [[] for i in range(graph.vertices)]

	for i in range(len(graph.edges)):
		(v_1, v_2) = graph.edges[i]

		out[v_2].append(i)
	
	return out

In [88]:
graph = StandardGraph(3, [
	(0,1),
    (1,2),
])

# graph = gen_planted_path(10, 0.05)

# max length
N = graph.vertices
K = N - 1

P = -1000

#binaries = range(floor(log2(K)) + 1)

tail_graph = make_tail_graph(graph, K)

# vertex_edges = make_vertex_edges(graph)
tail_vertex_edges = make_vertex_edges(tail_graph)

# binaries = []
# n=0
# while n <= floor(log2(K) + 0.01):
#     binaries.append(n)
#     n += 1

print(graph.edges)
print(tail_graph.edges)
print()

# for i in range(len(vertex_edges)):
#     print(f"{i} : {vertex_edges[i]}")
# print()

for i in range(len(tail_vertex_edges)):
    print(f"{i} : {tail_vertex_edges[i]}")


[(0, 1), (1, 2)]
[(0, 1), (1, 2), (0, 3), (1, 3), (2, 3), (3, 4)]

0 : [0, 2]
1 : [0, 1, 3]
2 : [1, 4]
3 : [2, 3, 4, 5]
4 : [5]


In [89]:


vertex_vars = Array.create('vertex', tail_graph.vertices, vartype='BINARY')
edge_vars = Array.create('edge', len(tail_graph.edges), vartype='BINARY')
initial_edge_vars = Array.create('init_edge', tail_graph.vertices, vartype='BINARY')
terminal_edge_vars = Array.create('term_edge', tail_graph.vertices, vartype='BINARY') 

#flow_vars = Array.create('flow', shape=(len(binaries), len(tail_graph.edges)), vartype='BINARY') #a variables
#contra_flow_vars = Array.create('contra_flow', shape=(len(binaries), len(tail_graph.edges)), vartype='BINARY') #b variables

# flow_vars_sums = [0 for _ in tail_graph.edges]
# contra_flow_vars_sums = [0 for _ in tail_graph.edges]

# for n in binaries:
# 	for i in range(len(tail_graph.edges)):
# 		flow_vars_sums[i] += 2**n * flow_vars[n][i]
# 		contra_flow_vars_sums[i] += 2**n * contra_flow_vars[n][i]


flow_vars = [LogEncInteger(f"flow_vars[{i}]", (0,K + 10)) for i in range(len(tail_graph.edges))]
initial_flow_vars = [LogEncInteger(f"initial_flow_vars[{i}]", (0,K + 10)) for i in range(tail_graph.vertices)] #assume ingoing
terminal_flow_vars = [LogEncInteger(f"terminal_flow_vars[{i}]", (0,K + 10)) for i in range(tail_graph.vertices)] #assume outgoing

contra_flow_vars = [LogEncInteger(f"contra_flow_vars[{i}]", (0,K + 10)) for i in range(len(tail_graph.edges))]
initial_contra_flow_vars = [LogEncInteger(f"initial_contra_flow_vars[{i}]", (0,K + 10)) for i in range(tail_graph.vertices)] #assume outgoing
terminal_contra_flow_vars = [LogEncInteger(f"terminal_contra_flow_vars[{i}]", (0,K + 10)) for i in range(tail_graph.vertices)] #assume ingoing


#(1)
candy_exp = sum(edge_vars[i] for i in range(len(graph.edges)))

#(2)
initial_node_exp = P * (1 - sum(initial_edge_vars[i] for i in range(graph.vertices)))**2

#(3)
terminal_node_exp = P * (1 - sum(terminal_edge_vars[i] for i in range(tail_graph.vertices)))**2

#(4)
fake_terminal_node_exp_1 = P * (vertex_vars[graph.vertices] - sum(edge_vars[i] for i in range( len(graph.edges), len(graph.edges) + graph.vertices)))**2

fake_terminal_node_exp_2 = P * (1 - vertex_vars[graph.vertices]) * vertex_vars[graph.vertices + 1]

#(5) and (6) can be together
real_node_deg_2_exp = 0
for i in range(tail_graph.vertices):
	temp1 = sum(edge_vars[tail_vertex_edges[i][j]] for j in range(len(tail_vertex_edges[i])))
	temp1 += initial_edge_vars[i] + terminal_edge_vars[i]

	# temp = sum(edge_vars[vertex_edges[i][j]] for j in range(len(vertex_edges[i])))
	# temp += initial_edge_vars[i] + terminal_edge_vars[i] + edge_vars[graph.vertices + i]

	print(temp1)
	# print("------")
	# print(temp)
	print()

	real_node_deg_2_exp += P * (2 * vertex_vars[i] - temp1)**2

#(6)
# tail_node_deg_2_exp = 0
# k = 0
# for i in range(graph.vertices + 1, tail_graph.vertices):
# 	tail_node_deg_2_exp += (2 * vertex_vars[i] - edge_vars[len(graph.edges) + k] - edge_vars[len(graph.edges) + k + 1] - terminal_edge_vars[i])**2
# 	#print((2 * vertex_vars[i] - edge_vars[len(graph.edges) + k] - edge_vars[len(graph.edges) + k + 1] - terminal_edge_vars[i])**2)
# tail_node_deg_2_exp = P * tail_node_deg_2_exp

#print(tail_node_deg_2_exp)

#(7)
labeling_edges_exp = P * sum([((K + 3) * edge_vars[i] - flow_vars[i] - contra_flow_vars[i])**2 for i in range(len(edge_vars))])

labeling_edges_exp += P * sum([((K + 3) * initial_edge_vars[i] - initial_flow_vars[i] - initial_contra_flow_vars[i])**2 for i in range(len(initial_edge_vars))])

labeling_edges_exp += P * sum([((K + 3) * terminal_edge_vars[i] - terminal_flow_vars[i] - terminal_contra_flow_vars[i])**2 for i in range(len(terminal_edge_vars))])

# skip (8) BAD

#(9)
# incomming_flow_exp = 0
# for i in range(tail_graph.vertices):
# 	temp_exp = (2 * K + 6) * vertex_vars[i]

# 	for j in tail_vertex_edges[i]:
# 		temp_exp -= flow_vars_sums[j]

# 		for k in tail_vertex_edges[i]:
# 			if k != j:
# 				temp_exp -= contra_flow_vars_sums[k]
	
# 	incomming_flow_exp += temp_exp**2
# incomming_flow_exp = P * incomming_flow_exp






((Binary('init_edge[0]') + Binary('term_edge[0]')) + Binary('edge[2]') + 0.000000 + Binary('edge[0]'))

((Binary('init_edge[1]') + Binary('term_edge[1]')) + Binary('edge[3]') + Binary('edge[1]') + 0.000000 + Binary('edge[0]'))

((Binary('init_edge[2]') + Binary('term_edge[2]')) + Binary('edge[4]') + 0.000000 + Binary('edge[1]'))

((Binary('init_edge[3]') + Binary('term_edge[3]')) + Binary('edge[5]') + Binary('edge[4]') + Binary('edge[3]') + 0.000000 + Binary('edge[2]'))

((Binary('init_edge[4]') + Binary('term_edge[4]')) + 0.000000 + Binary('edge[5]'))



In [90]:
total_exp = candy_exp + initial_node_exp + terminal_node_exp + fake_terminal_node_exp_1 + fake_terminal_node_exp_2 + real_node_deg_2_exp + labeling_edges_exp #+ incomming_flow_exp
# + tail_node_deg_2_exp # (6) is included in (5).

exp = -total_exp

model = exp.compile()
bqm = model.to_bqm()

sa = neal.SimulatedAnnealingSampler()
sampleset = sa.sample(bqm, num_reads=200, num_sweeps=100)
decoded_samples = model.decode_sampleset(sampleset)
best_sample = min(decoded_samples, key=lambda x: x.energy)
print(best_sample.energy)


print("Vertices:")
for i in range(tail_graph.vertices):
	print(f"vertex {i} : ", best_sample.sample[f"vertex[{i}]"])

print()
print("Edges:")
for i in range(len(tail_graph.edges)):
	print(f"Edge {i} : ", best_sample.sample[f"edge[{i}]"])

# print()
# print("Flow vars:")
# for i in range(len(tail_graph.edges)):
# 	for j in range(ceil(log2(K + 10))):
# 		print(f"Flow var {i} {j} : ", best_sample.sample[f"flow_vars[{i}][{j}]"])

print()

print("Initial flow vars:")
for i in range(tail_graph.vertices):	
	print(f"Initial flow var {i}: ", best_sample.subh[f"initial_flow_vars[{i}]"])

print("Flow vars:")
for i in range(len(tail_graph.edges)):	
	print(f"Flow var {i}: ", best_sample.subh[f"flow_vars[{i}]"])

print("Terminal flow vars:")
for i in range(tail_graph.vertices):
	print(f"Terminal flow var {i}: ", best_sample.subh[f"terminal_flow_vars[{i}]"])


print()

print("Contra initial flow vars:")
for i in range(tail_graph.vertices):	
	print(f"Initial contra flow var {i}: ", best_sample.subh[f"initial_contra_flow_vars[{i}]"])

print("Contra flow vars:")
for i in range(len(tail_graph.edges)):	
	print(f"Contra flow var {i}: ", best_sample.subh[f"contra_flow_vars[{i}]"])

print("Contra terminal flow vars:")
for i in range(tail_graph.vertices):	
	print(f"Terminal contra flow var {i}: ", best_sample.subh[f"terminal_contra_flow_vars[{i}]"])

print(tail_graph.edges)

1999.0
Vertices:
vertex 0 :  1
vertex 1 :  1
vertex 2 :  0
vertex 3 :  1
vertex 4 :  1

Edges:
Edge 0 :  1
Edge 1 :  0
Edge 2 :  0
Edge 3 :  1
Edge 4 :  0
Edge 5 :  0

Initial flow vars:
Initial flow var 0:  0.0
Initial flow var 0 (old):  0
Initial flow var 1:  0.0
Initial flow var 1 (old):  0
Initial flow var 2:  5.0
Initial flow var 2 (old):  8
Initial flow var 3:  0.0
Initial flow var 3 (old):  0
Initial flow var 4:  0.0
Initial flow var 4 (old):  0
Flow vars:
Flow var 0:  8
Flow var 1:  0
Flow var 2:  0
Flow var 3:  3
Flow var 4:  0
Flow var 5:  0
Terminal flow vars:
Terminal flow var 0:  0
Terminal flow var 1:  0
Terminal flow var 2:  0
Terminal flow var 3:  0
Terminal flow var 4:  3

Contra initial flow vars:
Initial contra flow var 0:  0
Initial contra flow var 1:  0
Initial contra flow var 2:  0
Initial contra flow var 3:  8
Initial contra flow var 4:  5
Contra flow vars:
Contra flow var 0:  0
Contra flow var 1:  0
Contra flow var 2:  0
Contra flow var 3:  2
Contra flow var 4: 